# Notebook C: Find GSM flux bounds by constraining the GSM with 13C-MFA
This Jupyter notebook focuses on using Flux Variability Analysis (FVA) to find the bounds of each reaction in a Genome Scale Model (GSM) 
when 13C-MFA reactions are used for GSM boundries.

### Objective
The primary objective of this analysis is to use the GSM to extrapolate the rates of reactions outside the scope of 13C-MFA. We explore the genome scale metabolic flux profiles when Y. lipolytica consumes glucose, glycerol, acetate, and oleic acid. 

### Important Functions
- `load_and_preprocess_data`: Loads genome-scale models and 13C-MFA data, and performs necessary preprocessing.
- `perform_CFBA`: Executes Constrained Flux Balance Analysis and prepares the data for comparison.
- `compare_flux_analysis`: Compares CFBA results with 13C-MFA data, highlighting key similarities and differences.
- `analyze_carbon_sources`: Analyzes the impact of different carbon sources on metabolic fluxes within the CFBA model.
- `integrate_strain_design`: Integrates StrainDesign predictions to enhance the understanding of the metabolic model.


### Load imports

In [1]:
import cobra
import straindesign as sd
import pandas as pd
import numpy as np
import sys
from straindesign import parse_linexpr, select_solver, parse_constraints


source_dir = '../src'
sys.path.append(source_dir)
from add_flux_column_to_13c_flux_df import add_flux_column_to_13c_flux_df
from add_fva_columns_to_13c_flux_df import add_fva_columns_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot
from make_rxn_constraint_string import make_rxn_constraint_string
from generate_flux_map import generate_flux_map

### Load the genome scale model

In [2]:
model = cobra.io.json.load_json_model('../genome_scale_models/iYLI647_corr_3.json')

### Rename some reactions to remove parentheses
This is because parentheses in reaction ids cause problems with StrainDesign

In [3]:
model.reactions.get_by_id('EX_glc(e)').id = 'EX_glc_e'
model.reactions.get_by_id('EX_glyc(e)').id = 'EX_glyc_e'
model.reactions.get_by_id('EX_ocdcea(e)').id = 'EX_ocdcea_e'
model.reactions.get_by_id('EX_h2o(e)').id = 'EX_h2o_e'
model.reactions.get_by_id('EX_h(e)').id = 'EX_h_e'
model.reactions.get_by_id('EX_nh4(e)').id = 'EX_nh4_e'
model.reactions.get_by_id('EX_o2(e)').id = 'EX_o2_e'
model.reactions.get_by_id('EX_pi(e)').id = 'EX_pi_e'
model.reactions.get_by_id('EX_so4(e)').id = 'EX_so4_e'

# print an example reaction
model.reactions.get_by_id('EX_glc_e')

Reaction identifier,EX_glc_e
Name,D Glucose exchange
Memory address,0x2d49e79a0
Stoichiometry,glc_D[e] <=> D_Glucose <=>
GPR,YALI0D01111g or YALI0D18876g or YALI0D00132g or YALI0B01342g or YALI0E23287g or YALI0B00396g or...
Lower bound,-10.0
Upper bound,1000.0


### Load 13C-MFA data

In [4]:
central_rxn_df = pd.read_excel('../data/13c_mfa/INCA model 12212023_summary_GR.xlsx', sheet_name='Full MFA Data')

# calculate the number of reactions in the 13C MFA that are mapped to the GSM
mapped_rxn_df = central_rxn_df.dropna(subset = ["reaction_ids"])

print(f'There are {len(mapped_rxn_df)} reactions in the 13C MFA that are mapped to the GSM')

central_rxn_df.head()

There are 43 reactions in the 13C MFA that are mapped to the GSM


,Pathway,ID,Equation,reaction_ids,Location on map,Glucose MFA Flux,Glucose MFA LB,Glucose MFA UB,Glycerol MFA Flux,Glycerol MFA LB,Glycerol MFA UB,Oleic Acid MFA Flux,Oleic Acid MFA LB,Oleic Acid MFA UB
0,uptake,gluc uptake,Glucose + ATP -> G6P,reverse_EX_glc_e,"(-1180, 1175)",100.0000,100.0000,100.0000,NaN,NaN,NaN,NaN,NaN,NaN
1,uptake,glyc uptake,Glycerol -> GLYC,reverse_GLYCt,"(-1376, 417)",NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN
2,uptake,R3.2,GLYC + ATP -> DHAP + 1.5*ATP,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN
3,uptake,OA uptake,OA + ATP -> 9*ACCOAcyt + 8*NADH + 12*ATP,OCDCEAt,"(-55, 111)",NaN,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000
4,glycolysis/gluconeogensis,R4 net,G6P <-> F6P,PGI,"(-1180, 960)",18.0496,9.7776,23.5421,-41.3947,-41.5761,-36.5906,-316.4244,-328.5227,-265.6116


# Glucose

### Make constraint string from 13C-MFA data for glucose

In [73]:
# can build this to include
mfa_reactions_to_use = [    
    # glycolysis
    'Glucose + ATP -> G6P',
    'G6P <-> F6P',
    'F6P + ATP <-> FBP',
    'FBP <-> DHAP + GAP',
    'DHAP <-> GAP',
    'GAP <-> G3P + ATP + NADH',
    'G3P <-> PEP',

    # pentose phosphate pathway
    'G6P -> PG6 + NADPH',
    'PG6 -> Ru5P + CO2 + NADPH',
    'Ru5P <-> R5P',
    'Ru5P <-> X5P',
    'X5P <-> GAP + TKC2',
    'E4P + TKC2 <-> F6P',
    'R5P + TKC2 <-> S7P',
    'GAP + TAC3 <-> F6P',
    'S7P <-> E4P + TAC3',

    # TCA cycle
    'PYRmit -> ACCOAmit + CO2 + NADH',
    'ACCOAmit + OAAmit -> CITmit',
    'CITmit <-> ICTmit',
    'ICTmit -> AKG + CO2 + NADH',
    'ICTmit -> AKG + CO2 + NADPH',
    'AKG -> 0.5*SUCmit + 0.5*SUCmit + CO2 + ATP + NADH',
    'SUCmit <-> FUMmit + 1.5*ATP',
    '0.5*FUMmit + 0.5*FUMmit <-> 0.5*MALmit + 0.5*MALmit',
    'MALmit <-> OAAmit + NADH',

    # glyoxylate shunt
    'ICTcyt -> GLYOXYcyt + SUCcyt',
    # 'ACCOAcyt + GLYOXYcyt -> MALcyt', # malate synthase causes problem
    'MALcyt <-> OAAcyt + NADH',

    # CO2 transport
    'CO2 -> CO2_EX',
]

constraint_strings = []

for mfa_reaction in mfa_reactions_to_use:
    # get the reaction row
    reaction_row = central_rxn_df[central_rxn_df['Equation'] == mfa_reaction]

    # get reaction IDs
    reaction_ids = reaction_row['reaction_ids'].values[0]

    # get the glucose lower bound
    glucose_lb = reaction_row['Glucose MFA LB'].values[0]

    # get the glucose upper bound
    glucose_ub = reaction_row['Glucose MFA UB'].values[0]

    # generate a constraint string
    constraint_string = make_rxn_constraint_string(reaction_ids, glucose_lb, glucose_ub)

    # add the constraint string to the list
    # constraint_strings.append(constraint_string)

# add biomass constraint
biomass_glucose_row = central_rxn_df[central_rxn_df['Pathway'] == 'biomass formation']
biomass_glucose_lb = biomass_glucose_row['Glucose MFA LB'].values[0]
biomass_glucose_ub = biomass_glucose_row['Glucose MFA UB'].values[0]

constraint_strings.append(f'biomass_glucose <= {biomass_glucose_ub}')
constraint_strings.append(f'biomass_glucose >= {biomass_glucose_lb}')

# block other biomass reactions
constraint_strings.append('biomass_oil = 0')
constraint_strings.append('biomass_C = 0')
constraint_strings.append('biomass_N = 0')

glucose_constraint_string = ', '.join(constraint_strings)

glucose_constraint_string

'biomass_glucose <= 13.0838, biomass_glucose >= 10.5561, biomass_oil = 0, biomass_C = 0, biomass_N = 0'

### Run Glucose pFBA with MFA constraints

In [74]:
# update the media to minimal medium with glucose as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 100
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

glucose_fba_solution = sd.fba(model, constraints=glucose_constraint_string, obj='biomass_glucose', obj_sense='maximize', pfba=1)

print()

max_glucose_biomass_flux = glucose_fba_solution['biomass_glucose']
PGI_flux = glucose_fba_solution['PGI']
print(f'Maximum biomass flux: {max_glucose_biomass_flux}.')
print(f'PGI flux: {PGI_flux}.')
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in glucose_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in glucose_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'glucose_GSM_flux': flux,
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
glucose_gsm_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
glucose_gsm_df = glucose_gsm_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
glucose_gsm_df = glucose_gsm_df.drop(columns=['absolute_flux'])

glucose_gsm_df.head()

100 EX_glc_e
10000 EX_h2o_e
10000 EX_h_e
10000 EX_nh4_e
10000 EX_o2_e
10000 EX_pi_e
10000 EX_so4_e

Maximum biomass flux: 0.0.
PGI flux: 0.0.
The number of active reactions in pFBA: 0


,reaction_id,reaction_name,full_reaction,glucose_GSM_flux
0,13BGH,Endo 1 3 beta glucan glucohydrase,13BDglcn[c] + h2o[c] --> glc_D[c],0.0
897,NTRLASE,nitrilase,2.0 h2o[c] + ind3acnl[c] --> ind3ac[c] + nh4[c],0.0
905,OCDCAt,Octadecanoate n C180 transport in via uniport,ocdca[e] --> ocdca[c],0.0
904,MCITDm,2 methylcitrate dehydratase mitochondrial,hcit[m] <=> b124tc[m] + h2o[m],0.0
903,OAAt2m,oxaloacetate transport mitochondrial,h[c] + oaa[c] <=> h[m] + oaa[m],0.0


### Run Glucose FVA with MFA constraints

In [75]:
# run FVA for 90% of biomass production on the GSM
glucose_fva_solution = sd.fva(
  model, 
  constraints=glucose_constraint_string,
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glucose_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glucose_gsm_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  lower_bound = glucose_fva_solution.loc[reaction_id, 'minimum']
  upper_bound = glucose_fva_solution.loc[reaction_id, 'maximum']
  

  fva_lower_bounds.append(lower_bound)
  fva_upper_bounds.append(upper_bound)
  

# add the upper and lower bounds to the dataframe
glucose_gsm_df['glucose_GSM_LB'] = fva_lower_bounds
glucose_gsm_df['glucose_GSM_UB'] = fva_upper_bounds

# save the dataframe to a csv file
glucose_gsm_df.to_csv('../results/gsm_fluxes/glucose_gsm_13C_fba.csv', index=False)

# display updated dataframe
glucose_gsm_df


The number of active reactions in FVA: 718


,reaction_id,reaction_name,full_reaction,glucose_GSM_flux,glucose_GSM_LB,glucose_GSM_UB
0,13BGH,Endo 1 3 beta glucan glucohydrase,13BDglcn[c] + h2o[c] --> glc_D[c],0.0,0.000000,147.109516
897,NTRLASE,nitrilase,2.0 h2o[c] + ind3acnl[c] --> ind3ac[c] + nh4[c],0.0,0.000000,-0.000000
905,OCDCAt,Octadecanoate n C180 transport in via uniport,ocdca[e] --> ocdca[c],0.0,0.000000,-0.000000
904,MCITDm,2 methylcitrate dehydratase mitochondrial,hcit[m] <=> b124tc[m] + h2o[m],0.0,2.852986,16.076313
903,OAAt2m,oxaloacetate transport mitochondrial,h[c] + oaa[c] <=> h[m] + oaa[m],0.0,-1000.000000,713.212193
...,...,...,...,...,...,...
448,EX_orn(e),Ornithine exchange,orn[e] -->,0.0,0.000000,17.723410
447,EX_ocdcya(e),octadecynoate n C182 exchange,ocdcya[e] -->,0.0,0.000000,-0.000000
446,EX_ocdcea_e,octadecenoate n C181 exchange,ocdcea[e] -->,0.0,0.000000,-0.000000
445,EX_ocdca(e),octadecanoate n C180 exchange,ocdca[e] -->,0.0,0.000000,-0.000000


In [76]:
# just look at rows with biomass in the reaction_id
biomass_rows = glucose_gsm_df[glucose_gsm_df['reaction_id'].str.contains('biomass')]
biomass_rows

,reaction_id,reaction_name,full_reaction,glucose_GSM_flux,glucose_GSM_LB,glucose_GSM_UB
1348,biomass_glucose,,0.6822317267700153 13BDglcn[c] + 0.55815800338...,0.0,10.5561,12.749805
156,biomass_N,biomass yarrowia nitrogen limiting,0.257113988 13BDglcn[c] + 0.09496 ala_L[c] + 0...,0.0,0.0000,-0.000000
155,biomass_C,biomass yarrowia carbon limiting,0.943396927 13BDglcn[c] + 0.567938715 ala_L[c]...,0.0,0.0000,-0.000000
1349,biomass_oil,,0.6822317267700153 13BDglcn[c] + 0.32336939465...,0.0,0.0000,-0.000000


In [77]:
# look at PFK and FBP reactions
reaction_df = glucose_gsm_df[glucose_gsm_df['reaction_id'].isin(['PFK', 'FBP'])]

reaction_df

,reaction_id,reaction_name,full_reaction,glucose_GSM_flux,glucose_GSM_LB,glucose_GSM_UB
572,FBP,fructose bisphosphatase,fdp[c] + h2o[c] --> f6p[c] + pi[c],0.0,0.0,294.219032
364,PFK,phosphofructokinase,atp[c] + f6p[c] --> adp[c] + fdp[c] + h[c],0.0,0.0,366.636073


### Add glucose GSM data to dataframe

In [ ]:
# add flux and bounds columns
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, glucose_gsm_df, 'glucose_GSM_flux')
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, glucose_gsm_df, 'glucose_GSM_LB', 'glucose_GSM_UB')

central_rxn_df.head(6)

### Plot glucose MFA bounds

In [ ]:
glucose_mfa_bounds_map = generate_flux_map(
    flux_df=central_rxn_df, 
    flux_column=['Glucose MFA LB', 'Glucose MFA UB'],
    title_string='Glucose 13C-MFA Bounds 2023-12-21',
    file_name='../figures/test_flux_map.png',
)

### Plot glucose new GSM bounds

In [ ]:
glucose_mfa_bounds_map = generate_flux_map(
    flux_df=central_rxn_df, 
    flux_column=['glucose_GSM_LB', 'glucose_GSM_UB'],
    title_string='Glucose GSM Constrained by MFA Bounds 2024-01-18',
    file_name='../figures/test_flux_map.png',
)

### Build glucose constraint string from 13C-MFA data

In [ ]:
# update the media to minimal medium with glucose as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 100
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

lower_bound_row = 'glucose_LB'
upper_bound_row = 'glucose_UB'

constraint_strings = []

# loop over rows in the central flux dataframe
for _, row in central_rxn_df.iterrows():  
    # get the GSM reaction mapping for this reaction
    reaction_ids = row['reaction_ids']
    pathway = row['pathway']

    # get the lower bound for this reaction
    lower_bound = row[lower_bound_row]
    upper_bound = row[upper_bound_row]

    # determine if reaction should be included in the constraint string
    reaction_ids_not_nan = not pd.isna(reaction_ids)
    pathway_not_transport = pathway != 'transport'
    bounds_are_not_nan = not pd.isna(lower_bound) and not pd.isna(upper_bound)
    

    if reaction_ids_not_nan and pathway_not_transport and bounds_are_not_nan:
        print(f'13C-MFA bounds: {lower_bound} - {upper_bound}')

        # make the constraint string
        constraint_string = make_rxn_constraint_string(reaction_ids, lower_bound, upper_bound)

        glucose_fba_solution = sd.fba(model, constraints=constraint_string, obj='biomass_glucose', obj_sense='maximize', pfba=1)

        # add the constraint string to the list
        constraint_strings.append(constraint_string)
        print(constraint_string)
        print(f"max biomass flux: {glucose_fba_solution['biomass_glucose']}")
        print()



full_constraint_string = ', '.join(constraint_strings)
print(len(full_constraint_string))
full_constraint_string 

### Calculate glucose GSM pFBA solution

### Calculate glucose GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
glucose_fva_solution = sd.fva(
  model, 
  constraints=f'EX_glc_e = -100.000, biomass_C >= {biomass_fraction * max_glucose_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glucose_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glucose_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = glucose_fva_solution.loc[reaction_id, 'maximum']
  lower_bound = glucose_fva_solution.loc[reaction_id, 'minimum']

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
glucose_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
glucose_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
glucose_gsm_fba_df.to_csv('../results/gsm_fluxes/glucose_gsm_13C_fba.csv', index=False)

# display updated dataframe
glucose_gsm_fba_df


### Add glucose pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, glucose_gsm_fba_df, 'glucose_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, glucose_gsm_fba_df, f'glucose_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Glycerol

### Calculate glycerol GSM pFBA solution

In [ ]:
# update the media to minimal medium with glycerol as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 0
medium['EX_glyc_e'] = 100
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

# run biomass-maximizing pFBA
glycerol_fba_solution = sd.fba(model, constraints='EX_glyc_e = -100.000', obj='biomass_C', obj_sense='maximize', pfba=1)

max_glycerol_biomass_flux = glycerol_fba_solution['biomass_C']
print(f'Maximum biomass flux: {max_glycerol_biomass_flux}.')
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in glycerol_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in glycerol_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': flux,
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
glycerol_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
glycerol_gsm_fba_df = glycerol_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
glycerol_gsm_fba_df = glycerol_gsm_fba_df.drop(columns=['absolute_flux'])

glycerol_gsm_fba_df.head()

### Calculate glycerol GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
glycerol_fva_solution = sd.fva(
  model, 
  constraints=f'EX_glyc_e = -100.000, biomass_C >= {biomass_fraction * max_glycerol_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glycerol_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glycerol_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = glycerol_fva_solution.loc[reaction_id, 'maximum']
  lower_bound = glycerol_fva_solution.loc[reaction_id, 'minimum']

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
glycerol_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
glycerol_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
glycerol_gsm_fba_df.to_csv('../results/gsm_fluxes/glycerol_gsm_13C_fba.csv', index=False)

# display updated dataframe
glycerol_gsm_fba_df

### Add glycerol pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, glycerol_gsm_fba_df, 'glycerol_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, glycerol_gsm_fba_df, f'glycerol_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Oleic Acid

### Calculate oleic acid GSM pFBA solution

In [ ]:
# update the media to minimal medium with oleic_acid as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 0
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 10 # this prevents overflow
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

# run biomass-maximizing pFBA
oleic_acid_fba_solution = sd.fba(model, constraints='EX_ocdcea_e = -10.000', obj='biomass_C', obj_sense='maximize', pfba=1)

max_oleic_acid_biomass_flux = oleic_acid_fba_solution['biomass_C']
print(f'Maximum biomass flux: {10 * max_oleic_acid_biomass_flux}.') # restore 100 input flux
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in oleic_acid_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in oleic_acid_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': 10 * flux, # restore 100 input flux
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
oleic_acid_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.drop(columns=['absolute_flux'])

oleic_acid_gsm_fba_df.head()

### Calculate oleic acid GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
oleic_acid_fva_solution = sd.fva(
  model, 
  constraints=f'EX_ocdcea_e = -10.000, biomass_C >= {biomass_fraction * max_oleic_acid_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in oleic_acid_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in oleic_acid_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = 10 * oleic_acid_fva_solution.loc[reaction_id, 'maximum'] # restore 100 input flux
  lower_bound = 10 * oleic_acid_fva_solution.loc[reaction_id, 'minimum'] # restore 100 input flux

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
oleic_acid_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
oleic_acid_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
oleic_acid_gsm_fba_df.to_csv('../results/gsm_fluxes/oleic_acid_gsm_13C_fba.csv', index=False)

# display updated dataframe
oleic_acid_gsm_fba_df


### Add oleic acid pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, oleic_acid_gsm_fba_df, 'oleic_acid_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, oleic_acid_gsm_fba_df, f'oleic_acid_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Save Data

### Save central flux data with pFBA data added

In [ ]:
# save the dataframe to a csv file
central_rxn_df.to_csv('../results/central_fluxes/13C_pfba.csv', index=False, encoding='utf-8-sig')

In [ ]:
# Code to delete later

# def build_13c_constraint_string(central_rxn_df, lower_bound_row, upper_bound_row):
    
#     # key node reactions to constrain
#     key_pathway_reactions = {
#         'EMP': 'GAPD',
#         'TCA': 'CSm',
#         'PPP': 'GND',
#     }
    
#     constraint_strings = []

#     for pathway, reaction_ids in key_pathway_reactions.items():
#         row = central_rxn_df.loc[central_rxn_df['reaction_ids'] == reaction_ids]
#         # get the lower bound for this reaction
#         lower_bound = row[lower_bound_row]
#         upper_bound = row[upper_bound_row]

#         constraint_string = make_rxn_constraint_string(reaction_ids, lower_bound, upper_bound)
#         constraint_strings.append(constraint_string)

#     full_constraint_string = ', '.join(constraint_strings)
#     return full_constraint_string

    

# build_13c_constraint_string(central_rxn_df, 'gluc LB', 'gluc UB')
